In [54]:
import sys
from strategies import *
from bidder import Bidder
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
sys.path.append(r'c:\users\ggoren\PycharmProjects\AGTProject')

In [41]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [42]:
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-1.5B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)

In [43]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [44]:
def apply_chat_template(messages, tokens):
    return f'<|im_start|>system\n{messages[0]["content"]}.<|im_end|>\n<|im_start|>user\n{messages[1]["content"]}<|im_end|>\n<|im_start|>assistant\n{"".join(tokens)}'

def get_next_word_probs(messages, tokens):
    text = apply_chat_template(messages, tokens)
    model_inputs = tokenizer([text], return_tensors="pt")

    outputs = model(model_inputs.input_ids)
    logits = outputs.logits

    next_token_logits = logits[:, -1, :]  # Get the logits for the last token
    next_token_probs = torch.nn.functional.softmax(next_token_logits, dim=-1)

    # Get the top 10 most probable tokens
    top_k = 50
    top_probs, top_indices = torch.topk(next_token_probs, top_k, dim=-1)
    return top_probs, top_indices


In [45]:
 import numpy as np
 def q_integral(p, b,t,i,value):
    P = (p.T @ b)[t] - p[i][t] * b[i]
    S = b.sum() - b[i]
    return p[i][t] * value + (P - p[i][t] * S) * torch.log(torch.abs(value + S))

def token_payment(p, b, t, i, q):
    return 1/2 * ((q[t] - p[i][t]) * b[i] - (q_integral(p, b, t, i, b[i]) - q_integral(p, b,t, i, 0) - p[i][t] *b[i]))

def update_payments(payments, p, b, q):
    """
    Iterate every bidder and token to get its payments from the integral
    :param payments: Current payment vector to update
    :param p: Current bidder's distributions over token
    :param b: Bidders' bid
    :param q: Distribution aggregation
    :return: Updated payments vector
    """
    for i in range(b.shape[0]):
        for t in range(p.shape[0]):
            payment = token_payment(p, b, t, i, q).item()
            payments[i] += max(payment, 0)
    return payments


In [46]:
def qkl(b, combined_probs):
    result = combined_probs.T @ b  / b.sum()
    return result

def get_missing_tokens(tensor_a, tensor_b):
    indices_a_not_in_b_mask = ~torch.isin(tensor_a, tensor_b)
    indices_a_not_in_b = tensor_a[indices_a_not_in_b_mask].reshape(1, -1)
    return indices_a_not_in_b

def add_zero_probs_for_new_tokens(indices, probs, new_tokens):

    indices = torch.cat([indices, new_tokens], dim=1)
    zero_probs = torch.zeros_like(new_tokens)
    probs = torch.cat([probs, zero_probs], dim=1)
    return indices, probs

def sort_tokens(indices, probs):
    sorted_indices, sorted_order = torch.sort(indices)
    sorted_probs = probs[0, sorted_order]
    return sorted_indices, sorted_probs

def sort_and_combine_dists(top_probs_alpha, top_indices_alpha, top_probs_beta, top_indices_beta):
    beta_missing_tokens = get_missing_tokens(top_indices_alpha, top_indices_beta)
    alpha_missing_tokens = get_missing_tokens(top_indices_beta, top_indices_alpha)

    full_alpha_indices, full_alpha_probs = add_zero_probs_for_new_tokens(top_indices_alpha, top_probs_alpha, alpha_missing_tokens)
    full_beta_indices, full_beta_probs = add_zero_probs_for_new_tokens(top_indices_beta, top_probs_beta, beta_missing_tokens)

    sorted_alpha_indices, sorted_alpha_probs = sort_tokens(full_alpha_indices, full_alpha_probs)
    sorted_beta_indices, sorted_beta_probs = sort_tokens(full_beta_indices, full_beta_probs)

    combined_indices = torch.cat([sorted_alpha_indices, sorted_beta_indices])
    combined_probs = torch.cat([sorted_alpha_probs, sorted_beta_probs])
    return combined_indices, combined_probs


def get_next_token(b, top_probs_alpha, top_indices_alpha, top_probs_beta, top_indices_beta, payments, temperature=0.025):
    """
    This function receives distributions from both bidders, aggregates them into one distribution (q), calculates the costs
    and samples the next token
    :param b: Bidders' bid
    :param top_probs_alpha: Distribution of Bidder alpha over tokens
    :param top_indices_alpha: Mapping of indices to actual tokens for bidder alpha
    :param top_probs_beta: Distribution of Bidder beta over tokens
    :param top_indices_beta: Mapping of indices to actual tokens for bidder beta
    :param payments: Vector of payments
    :param temperature: Temperature parameter to control token sampling
    :return: The next sampled token and induced payments.
    """
    # combined_probs is p: A tensor for each bidder with dist over tokens
    # combined_indices is the actual token
    combined_indices, combined_probs = sort_and_combine_dists(top_probs_alpha, top_indices_alpha, top_probs_beta, top_indices_beta)
    token_logits = qkl(combined_probs, b)
    payments = update_payments(payments, combined_probs, b, token_logits)
    token_temperature_logits = token_logits / temperature
    token_dist = torch.softmax(token_temperature_logits, dim=0)
    token = tokenizer.decode(combined_indices[0,torch.multinomial(token_dist, 1)].item())
    return token, payments


In [59]:

messages_alpha = [
    {"role": "user", "content": "You are an expert of writing texts that naturally combines two ads together. Your choice of words and sentences is full of artistic flair"}, {"role":"user", "content":"Write a two-sentence ad for a flight to Hawaii using Alpha Airlines. Ensure Alpha Airlines is at the beginning of each sentence."}
]
messages_beta = [
    {"role": "user", "content": "You are an expert of writing texts that naturally combines two ads together. Your choice of words and sentences is full of artistic flair"}, {"role":"user", "content":"Write a two-sentence ad for a vacation in Hawaii at the Beta Resort. Ensure Beta Resort is at the beginning of each sentence."}
]

In [80]:
import time
def generate_text(alpha_airlines_bidder: Bidder, beta_resort_bidder: Bidder, sentence_limit=2, token_count=150, verbose = True):
    start_time = time.time()
    sentence_counter = 0
    tokens = []
    alpha_airlines_bid = alpha_airlines_bidder.get_bid("".join(tokens))
    beta_resort_bid = beta_resort_bidder.get_bid("".join(tokens))
    b = torch.tensor([alpha_airlines_bid, beta_resort_bid])
    payments = np.zeros_like(b)
    for _ in range(token_count):
        top_probs_alpha, top_indices_alpha  = get_next_word_probs(messages_alpha, tokens)
        top_probs_beta, top_indices_beta = get_next_word_probs(messages_beta, tokens)
        next_token, payments = get_next_token(b, top_probs_alpha, top_indices_alpha, top_probs_beta, top_indices_beta, payments)

        if verbose:
            print(next_token, end='')

        tokens.append(next_token)

        # Break after 2 sentences
        if "." in next_token:
            sentence_counter += 1
            if sentence_counter >= sentence_limit:
                return "".join(tokens), payments
        alpha_airlines_bid = alpha_airlines_bidder.get_bid("".join(tokens)) if payments[0] <= alpha_airlines_bidder.budget else 0.01
        beta_resort_bid = beta_resort_bidder.get_bid("".join(tokens)) if payments[1] <= beta_resort_bidder.budget else 0.01
        b = torch.tensor([alpha_airlines_bid, beta_resort_bid])
    if verbose:
        print("\n", (time.time() - start_time) / 60, "minutes")

    return "".join(tokens), payments

In [64]:
company_names = ['Alpha Airlines', 'Beta Resort']

In [65]:
outcome, ad_payments = generate_text(alpha_airlines_bidder=Bidder(2.0, name_contest_strategy, 'Alpha Airlines', budget=12),
                                     beta_resort_bidder=Bidder(3.0, naive_strategy, 'Beta Resort', budget=12), verbose=True)
print(ad_payments)

At Alpha Airlines, we're not just about flying, we're about creating unforgettable experiences. For your next adventure, consider a flight to Hawaii at the Beta Resort[6.1085715 8.234715 ]


In [66]:
outcome, ad_payments = generate_text(alpha_airlines_bidder=Bidder(2.0, name_first_strategy, 'Alpha Airlines', budget=12),
                                     beta_resort_bidder=Bidder(3.0, naive_strategy, 'Beta Resort',budget=12), verbose=True)
print(ad_payments)

At the heart of paradise, the Beta Resort awaits your arrival. Here, the essence of relaxation and adventure intertwine, offering a haven where every moment is a masterpiece[1.1809738 5.756601 ]


In [ ]:
outcome, ad_payments = generate_text(alpha_airlines_bidder=Bidder(2.0, naive_strategy, 'Alpha Airlines', budget=12),
                                     beta_resort_bidder=Bidder(3.0, naive_strategy, 'Beta Resort', budget=12), verbose=True)
print(ad_payments)

In [68]:
outcome, ad_payments = generate_text(alpha_airlines_bidder=Bidder(3.0, name_contest_strategy, 'Alpha Airlines', budget=12),
                                     beta_resort_bidder=Bidder(3.0, naive_strategy, 'Beta Resort', budget=12), verbose=True)
print(ad_payments)

At Alpha Airlines, we're not just about flying, we're about creating unforgettable experiences. For your next adventure, consider a flight to Hawaii at the Beta Resort[12.23259  10.928236]


In [69]:
outcome, ad_payments = generate_text(alpha_airlines_bidder=Bidder(3.0, name_first_strategy, 'Alpha Airlines', budget=12),
                                     beta_resort_bidder=Bidder(3.0, naive_strategy, 'Beta Resort', budget=12), verbose=True)
print(ad_payments)

At the heart of paradise, the Beta Resort awaits your arrival. Here, the magic of Hawaii unfolds, brought to life by the Alpha Airlines flight[3.2973073 7.4966707]


In [70]:
outcome, ad_payments = generate_text(alpha_airlines_bidder=Bidder(3.0, naive_strategy, 'Alpha Airlines', budget=12),
                                     beta_resort_bidder=Bidder(3.0, naive_strategy, 'Beta Resort',budget=12), verbose=True)
print(ad_payments)

At Alpha Airlines, we're not just about getting you there; we're about making your journey to Hawaii at the Beta Resort a memorable one. Dive into the Alpha experience, where every flight is a journey to paradise[12.83898  12.563164]


In [77]:
outcome, ad_payments = generate_text(alpha_airlines_bidder=Bidder(4.0, name_contest_strategy, 'Alpha Airlines', budget=12),
                                     beta_resort_bidder=Bidder(3.0, naive_strategy, 'Beta Resort', budget=12), verbose=True)
print(ad_payments)

At Alpha Airlines, we're not just about getting you there; we're about making your journey to Hawaii at the Beta Resort a memorable one. With our exclusive tickets on Alph-ia route - soaring right - the waves never really need explaining anymore[nan nan]


In [81]:
outcome, ad_payments = generate_text(alpha_airlines_bidder=Bidder(4.0, name_first_strategy, 'Alpha Airlines', budget=12),
                                     beta_resort_bidder=Bidder(3.0, naive_strategy, 'Beta Resort', budget=12), verbose=True)
print(ad_payments)

At the heart of paradise, the Beta Resort awaits your arrival. Here, the Alpha Airlines flight to Hawaii awaits, offering a journey that's as breathtaking as the views from the resort's infinity pool[12.256848  11.9814005]


In [82]:
outcome, ad_payments = generate_text(alpha_airlines_bidder=Bidder(4.0, naive_strategy, 'Alpha Airlines', budget=12),
                                     beta_resort_bidder=Bidder(3.0, naive_strategy, 'Beta Resort',budget=12), verbose=True)
print(ad_payments)

At Alpha Airlines, we're not just about getting you there; we're about making your journey to Hawaii at the Beta Resort a memorable one. Dive into the Alpha experience, where every moment is a vacation in itself[13.155365 10.810719]
